In [1]:
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
import gc
import time
gc.enable()
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import re
import requests
import folium
import branca.colormap as cm
import geopy
from tqdm import tqdm_notebook as tqdm
import json
import os
import geojson

warnings.filterwarnings('ignore')

In [2]:
from shapely.geometry import shape, Point
from shapely.ops import cascaded_union

## Load Chicago area geojson

In [3]:
%%time
# Get Chicago area data
if not os.path.exists('../geo_data/chicago.geojson'):
    print("Creating geojson data for area of Chicago")
    # Load zip geojson
    with open('../geo_data/Boundaries_ZIPCodes.geojson', 'r') as p:
        zipcodes = json.load(p)
        
    # Merge all polygons
    polygons = []
    for feature in zipcodes['features']:
        polygon = shape(feature['geometry'])
        polygons.append(polygon)

    chicago_area = cascaded_union(polygons)
    
    # Save to geojson file
    with open('../geo_data/chicago.geojson', 'w') as f:
        geojson.dump(chicago_area, f)
else:
    print("Loading geojson data for Chicago")
    with open('../geo_data/chicago.geojson', 'r') as f:
        chicago_area = geojson.load(f)

Loading geojson data for Chicago
CPU times: user 52 ms, sys: 5.93 ms, total: 57.9 ms
Wall time: 68.2 ms


## Load Evanston data

In [4]:
%%time
# Get Evanston area data
if not os.path.exists('../geo_data/evanston.geojson'):
    print("Creating geojson data for area of Evanston")
    # Load zip geojson
    with open('../geo_data/Zoning_Districts.geojson', 'r') as p:
        zones = json.load(p)
        
    # Merge all polygons
    polygons = []
    for feature in zones['features']:
        polygon = shape(feature['geometry'])
        polygons.append(polygon)

    evanston_area = cascaded_union(polygons)
    
    # Save to geojson file
    with open('../geo_data/evanston.geojson', 'w') as f:
        geojson.dump(evanston_area, f)
else:
    print("Loading geojson data for Evanston")
    with open('../geo_data/evanston.geojson', 'r') as f:
        evanston_area = geojson.load(f)

Loading geojson data for Evanston
CPU times: user 10.8 ms, sys: 2.94 ms, total: 13.7 ms
Wall time: 15.3 ms


## Get data for Oak Park

In [5]:
## get cook county data
def _get_cook_county_data():
    if not os.path.exists('../geo_data/cook_county.json'):
        print("Collect Cook County data from web API.")
        res = requests.get('https://datacatalog.cookcountyil.gov/api/views/nr9b-xw4j/rows.json?accessType=DOWNLOAD')
        json_f = res.json()['data']
        with open('../geo_data/cook_county.json', 'w') as f:
            json.dump(json_f, f)
    else:
        with open('../geo_data/cook_county.json', 'r') as f:
            json_f = json.load(f)    
    return json_f

In [6]:
%%time
import shapely.wkt
if not os.path.exists('../geo_data/oak_park.geojson'):
    print("Creating geojson data for area of Oak Park")
    ccjs = _get_cook_county_data()
    for i in range(len(ccjs)):
        city = ccjs[i][-7]
        if 'oak park' in city.lower():
            opraw = ccjs[i]
            
    oak_park_area = shapely.wkt.loads(opraw[8])
    with open('../geo_data/oak_park.geojson', 'w') as f:
        geojson.dump(oak_park_area, f)
else:
    print("Loading geojson data for area of Oak Park")
    with open('../geo_data/oak_park.geojson', 'r') as f:
        oak_park_area = geojson.load(f)

Loading geojson data for area of Oak Park
CPU times: user 3.22 ms, sys: 2.18 ms, total: 5.4 ms
Wall time: 12 ms


## Collecting station data

In [7]:
%%time
# Load from preprocessed data
sd = pd.read_feather('../data/final_station_data.feather')

CPU times: user 34.4 ms, sys: 36.4 ms, total: 70.8 ms
Wall time: 226 ms


In [8]:
sd.iloc[0].city_Chicago

1.0

In [9]:
sd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618 entries, 0 to 617
Data columns (total 12 columns):
index            618 non-null int64
id               618 non-null int64
lon_ave          618 non-null float64
lat_ave          618 non-null float64
dp_max           618 non-null float64
dp_min           618 non-null float64
online_month     618 non-null float64
online_day       618 non-null float64
online_year      618 non-null float64
city_Chicago     618 non-null uint8
city_Evanston    618 non-null uint8
city_Oak_Park    618 non-null uint8
dtypes: float64(7), int64(2), uint8(3)
memory usage: 45.3 KB


## Plot stations on the map

Choose colors based on this [ref](https://github.com/gravitystorm/openstreetmap-carto/issues/3641)

List of colors:
```python
class Icon(MacroElement):
    """
    Creates an Icon object that will be rendered
    using Leaflet.awesome-markers.

    Parameters
    ----------
    color : str, default 'blue'
        The color of the marker. You can use:

            ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
             'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
             'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
             'gray', 'black', 'lightgray']

    icon_color : str, default 'white'
        The color of the drawing on the marker. You can use colors above,
        or an html color code.
```

In [42]:
from folium import GeoJson
from folium.map import Popup

divvy_station_map = folium.Map(location = [41.90, -87.64], tiles='Stamen Toner', zoom_start = 13.5)


for i in range(sd.shape[0]):
    long = sd.iloc[i].lon_ave
    lat = sd.iloc[i].lat_ave
    point = Point(long, lat)

    folium.Marker([lat, long], 
                   popup=Popup(
                       'station_id: {}\n longitude: {:.3f}\n lattitude: {:.3f}'.format(
                           int(sd.iloc[i].id), float(long), float(lat)
                       ),
                       max_width=1
                              ),
                   icon=folium.Icon(icon='bicycle', color='beige', prefix='fa')
                ).add_to(divvy_station_map)

divvy_station_map
#divvy_station_map.save('bikestations.html')

## Plot stations density

Folium API for [Choropleth](https://python-visualization.github.io/folium/modules.html#module-folium.features)

In [11]:
# Load zip geojson
with open('../geo_data/Boundaries_ZIPCodes.geojson', 'r') as p:
    zipcodes = json.load(p)

# Create a dictionary with (key: zipcodes, value: polygon)
polygons_dict = {}
for feature in zipcodes['features']:
    polygon = shape(feature['geometry'])
    polygons_dict[feature['properties']['zip']] = polygon

In [12]:
# Count number of stations in each zipcode zone
zip_dict = {}
for lon, lat in zip(np.array(sd.lon_ave), np.array(sd.lat_ave)):
    p = Point(lon, lat)
    for z, po in polygons_dict.items():
        if po.contains(p):
            if z in zip_dict:
                zip_dict[z] += 1
            else:
                zip_dict[z] = 1
            break
             

In [13]:
# Convert dict data into a dataframe
df = pd.DataFrame()

df['zipcode'] = zip_dict.keys()
df['count'] = zip_dict.values()

In [43]:
m = folium.Map(location = [41.85, -87.65], tiles='cartodbpositron', zoom_start = 10, width=500)

folium.Choropleth(
    geo_data=zipcodes,
    name='num_bike_stations',
    data=df,
    columns=['zipcode', 'count'],
    key_on='feature.properties.zip',
    bins=8,
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='white',
    legend_name='Number of bike stations'
).add_to(m)

m
#m.save('bikestation_density.html')